# Klassifikation mit strukturierten Daten

## Beispiel Iris-Datenset

### Vektorisierung

Einige Datensets sind bereits schon in `scikit-learn` eingebaut. Du brauchst dich nicht mehr darum zu kümmern, diese
* herunterzuladen,
* zu verifizieren und
* zu vektorisieren

Das [Iris-Datenset](https://en.wikipedia.org/wiki/Iris_flower_data_set) ist ein *Klassiker*:

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/220px-Kosaciec_szczecinkowaty_Iris_setosa.jpg) ![image](https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/220px-Iris_versicolor_3.jpg) ![image](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/220px-Iris_virginica.jpg)

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

Leider in einem etwas sperrigen Format

In [ ]:
iris

Besser als `DataFrame` mit `pandas`

In [ ]:
import pandas as pd
idf = pd.DataFrame(iris["data"], columns=["Sepal Length", "Sepal Width", "Petal Length", "Petal Width"])
idf["target"] = iris["target"]
idf["name"] = [iris["target_names"][target] for target in iris["target"]]
idf

*Five Number Summaries*

In [ ]:
idf.describe()

Verteilungen als Histogramm plotten

In [ ]:
idf["Petal Width"].plot.hist()

Überlegung, welche Features sich besonders zur Differenzierung eignen:

In [ ]:
idf.plot.scatter(x="Sepal Length", y="Sepal Width", c="target", cmap="Set1")

Oder alle auf einmal? Hier erkennst du mögliche Korrelationen oder in welchen Dimensionen die Objekte sich am deutlichsten unterscheiden:

In [ ]:
import seaborn as sns
sns.pairplot(idf.drop(columns=["target"]), hue="name")

### Iris-Klassifikation

Zum Training nutzt du die Iris-Daten, von denen du das Ergebnis schon kennst. Als Modell verwendest du eine sog. [Support Vector Machine](https://de.wikipedia.org/wiki/Support_Vector_Machine). Diese funktioniert sehr effizient:

In [ ]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
svm.fit(iris['data'], iris['target'])

Jetzt kannst du aus den Daten vorhersagen lassen, zu welcher Iris-Sorte eine Pflanze gehört, wenn du nur die Messwerte kennst.

In [ ]:
svm.predict(iris['data'])

Wie vergleicht sich das mit den echten Werten?

In [ ]:
svm.predict(iris['data']) == iris['target']

Eigentlich sieht das ganz gut aus, aber es gibt mehrere Probleme:
* Kann man die Performance des Klassifikators *messen*?
* Hat der Klassifikator evtl. nur (fast) alles auswendig gelernt oder kann er tatsächlich *abstrahieren*?

### Trainings-Test-Split

Um das zu beheben, kannst du die vorklassifizierte Datenmenge in zwei Teile zerlegen: eine *Trainingsmenge*, die nur dem Training des Klassifikators dient und eine davon unabhängige *Testmenge*, mit der der Klassifikator beweisen kann, wie gut er abstrahieren kann.

`scikit-learn` stellt dafür eine Funktion `train_test_split` zur Verfügung. Der `random_state` dient der Reproduzierbarkeit, mit `test_size` kann man das Verhältnis der beiden Mengengrößen wählen.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size = 0.25, random_state = 42)

Wieder trainierst du die SVM als Klassifikator:

In [ ]:
svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
svm.fit(X_train, y_train)

Dieses Mal sagst du die Spezies für die Testdaten vorher:

In [ ]:
svm.predict(X_test) == y_test

Zwei Werte sind falsch vorhergesagt. Welche das sind, findest du in der [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix):

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, svm.predict(X_test))

Auf der Diagonalen stehen dort die richtig vorgergesagten Werte, die `2` außerhalb der Diagonale zeigt dir falsch vorhergesagte Ergebnisse. In diesem Fall wäre das richtige Ergebnis die Klasse `2` gewesen (2. Spalte), stattdessen wurde `3` vorhergesagt (3. Zeile).

Oftmals möchtest du die Genauigkeit der Klassifikation messen. Dazu dienen gleich zwei Größen, nämlich die Precision (Spezifizität) und der Recall (Sensitivität):

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, svm.predict(X_test)))

Im Gegensatz zur häufig verwendeten *Accuracy* können Precision und Recall auch mit *schiefen* Verteilungen umgehen.